In [42]:
!pip install --upgrade geneva lancedb google-genai kubernetes "ray[default]" rerankers -q

/opt/conda/lib/python3.10/site-packages/lancedb/__init__.py:238: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/lance/__init__.py:168: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [46]:
#!/bin/bash
!curl -L -o fashion-product-images-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/paramaggarwal/fashion-product-images-dataset

!unzip -q fashion-product-images-dataset.zip 
#### TOY DATASET ###3
#!/bin/bash
#!curl -L -o fashion-product-images-small.zip\
#  https://www.kaggle.com/api/v1/datasets/download/paramaggarwal/fashion-product-images-small
# !unzip -q fashion-product-images-small.zip -d fashion-dataset/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 23.0G  100 23.0G    0     0   221M      0  0:01:46  0:01:46 --:--:--  209M


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!sudo rm -r db

In [1]:
import time
t1 = time.time()

In [ ]:
import os
import io
import geneva
import lancedb
import concurrent.futures


import pandas as pd
import geneva as gv
import pyarrow as pa

from pathlib import Path
from PIL import Image
from google import genai


import torch
from transformers import CLIPProcessor, CLIPModel

# === CONFIG ===
os.environ["API_KEY"] = os.getenv("GEMINI_API_KEY") or "..."


IMG_DIR = Path("fashion-dataset/images")
STYLE_CSV = Path("fashion-dataset/styles.csv")
DB_PATH = "./db"
TABLE_NAME = "products"
INSERT_FRAG_SIZE = 10000

In [3]:
df = pd.read_csv(STYLE_CSV, on_bad_lines='skip')
df = df.dropna(subset=["id", "productDisplayName"])    
df = df.drop_duplicates(subset=["id"], keep="first")    
df = df.dropna(subset=["id", "productDisplayName"]) # set to 100 for testing
print(len(df))
def generate_rows(df, img_dir):
    for _, row in df.iterrows():
        img_path = img_dir / f"{row['id']}.jpg"
        if not img_path.exists():
            continue
        with open(img_path, "rb") as f:
            yield {
                "id": int(row["id"]),
                "description": row["productDisplayName"],
                "image_bytes": f.read()
            }

db = lancedb.connect(DB_PATH)
if TABLE_NAME in db.table_names():
    db.drop_table(TABLE_NAME)
    
data_stream = generate_rows(df, IMG_DIR)
table = None

rows = []
for row in data_stream:
    rows.append(row)
    if len(rows) == INSERT_FRAG_SIZE:
        if table:
            table.add(rows)
        else:
            table = db.create_table(TABLE_NAME, data=rows)
        rows = []
if rows:
    table.add(rows)
    
len(table)

44417


44412

In [ ]:

# === FEATURE ENGINEERING with Geneva UDFs ===
table = gv.connect(DB_PATH).open_table(TABLE_NAME)

# Simple tag extractors
@gv.udf
def color_tags(description: str)-> str:
    colors = ["black", "white", "red", "blue", "green", "yellow", "pink", "brown"]
    return " , ".join([c for c in colors if c in description.lower()])



@gv.udf(data_type=pa.string())
def occasion_tagger(batch: pa.RecordBatch) -> pa.Array:
    _gemini = genai.Client(api_key="...")
    descriptions = batch.column("description").to_pylist()

    def call(desc: str) -> str:
        prompt = (
            f"Based on the following product description, describe the most suitable "
            f"occasion(s) to wear this dress in ≤25 words:\n\n{desc}"
        )
        resp = _gemini.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=prompt,
            config={"temperature": 0.0},
        )
        return resp.text.strip() if resp.text else None

    with concurrent.futures.ThreadPoolExecutor(max_workers=80) as exec:
        occasions = list(exec.map(call, descriptions))

    return pa.array(occasions, type=pa.string())

    

@gv.udf(data_type=pa.string())
def summarizer(batch: pa.RecordBatch) -> pa.Array:
    _gemini = genai.Client(api_key="...")
    descriptions = batch.column("description").to_pylist()
    
    def call(desc: str) -> str:
        resp = _gemini.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=f"Summarize in ≤25 words:\n{desc}",
            config={"temperature": 0.0},
        )
        return resp.text.strip() if resp.text else None
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=80) as exec:
        summaries = list(exec.map(call, descriptions))
    
    return pa.array(summaries, type=pa.string())

In [5]:
table.add_columns({
    "color_tags": color_tags,
    "occasion": occasion_tagger,
    "summary": summarizer
})

In [6]:
db = geneva.connect(DB_PATH)
if TABLE_NAME in db.table_names():
    table = db[TABLE_NAME]

In [7]:
table.schema

id: int64
description: string
image_bytes: binary
color_tags: string
  -- field metadata --
  virtual_column.platform.python_version: '3.10.18'
  virtual_column.platform.system: 'Linux'
  virtual_column: 'true'
  virtual_column.udf: '_udfs/682cde9a5fae61d4a646676e925c720ccf8752b2aa6a' + 20
  virtual_column.udf_inputs: '["description"]'
  virtual_column.platform.arch: 'x86_64'
  virtual_column.udf_name: 'color_tags'
  virtual_column.udf_backend: 'DockerUDFSpecV1'
occasion: string
  -- field metadata --
  virtual_column: 'true'
  virtual_column.platform.system: 'Linux'
  virtual_column.udf_backend: 'DockerUDFSpecV1'
  virtual_column.udf: '_udfs/78df167965d047a24e1d56bdb4860a4db5559020d570' + 20
  virtual_column.platform.arch: 'x86_64'
  virtual_column.platform.python_version: '3.10.18'
  virtual_column.udf_inputs: 'null'
  virtual_column.udf_name: 'occasion_tagger'
summary: string
  -- field metadata --
  virtual_column.udf: '_udfs/4b4610b4dcc9cf7b8a8f8176427e26940a6d0ae5bf18' + 20
  vir

In [8]:
import math

CONCURRENCY = 9 # REDUCE to 4 ON COLAB 

BATCH_SIZE = 5000

In [9]:
table.backfill("color_tags", batch_size=BATCH_SIZE, concurrency=CONCURRENCY)

Cluster nodes provisioned: |           0 [00:00]

geneva compute context not ready


Workers scheduled: |           0 [00:00]

[2025-07-28T10:32:32Z WARN  lance::dataset::write::insert] No existing dataset at /home/jupyter/semantic_router/db/geneva_jobs.lance, it will be created
/opt/conda/lib/python3.10/site-packages/lancedb/__init__.py:238: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/lance/__init__.py:168: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/lancedb/__init__.py:238: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/lance/__init__.py:168: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
2025-07-28 10:32:37,770	INFO worker.py:1918 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Workers started: 0it [00:00, ?it/s]

Batches checkpointed: 0it [00:00, ?it/s]

Fragments written: 0it [00:00, ?it/s]

(run_ray_add_column_remote pid=2287045) [2025-07-28T10:32:42Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:32:42Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:32:42Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:32:43Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:32:43Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable 

'e00a3a97-7b06-4cd4-9137-819ad0fd8cc1'

(run_ray_add_column_remote pid=2287045) [2025-07-28T10:38:04Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:38:04Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:38:04Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:38:04Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable yet, please use with caution.
(run_ray_add_column_remote pid=2287045) [2025-07-28T10:38:04Z WARN  lance::dataset::transaction] Building manifest with DataReplacement operation. This operation is not stable 

In [10]:
table.backfill("occasion", batch_size=BATCH_SIZE, concurrency=CONCURRENCY, where="1=1")

Cluster nodes provisioned: |           0 [00:00]

Workers scheduled: |           0 [00:00]

Workers started: 0it [00:00, ?it/s]

Batches checkpointed: 0it [00:00, ?it/s]

Fragments written:   0%|          | 0/5 [00:00<?, ?it/s]

'95b61daa-0467-4954-82d9-262884a3e00f'

In [11]:
table.backfill("summary", batch_size=BATCH_SIZE, concurrency=CONCURRENCY, where="1=1")

Cluster nodes provisioned: |           0 [00:00]

Workers scheduled: |           0 [00:00]

Workers started: 0it [00:00, ?it/s]

Batches checkpointed:   0%|          | 0/9 [00:00<?, ?it/s]

Fragments written:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
table.search().limit(3).to_pandas()

In [ ]:
import torchvision.transforms as transforms
import pyarrow as pa
import geneva as gv
import torch
from transformers import AutoTokenizer, AutoModel

@gv.udf(data_type=pa.list_(pa.float32(), 512),
       cuda=True
       )
class EmbedImage:
    def __init__(self):
        self.ready = False

    def setup(self):
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").cuda()
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                (0.48145466, 0.4578275, 0.40821073),
                (0.26862954, 0.26130258, 0.27577711)
            )
        ])
        self.ready = True


    def __call__(self, batch: pa.RecordBatch) -> pa.Array:
        if not self.ready:
            self.setup()
            
        img_bytes = batch.column("image_bytes").to_pylist()
        
        def preprocess(b):
            img = Image.open(io.BytesIO(b)).convert("RGB")
            return self.transform(img)

        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as exe:
            tensors = list(exe.map(preprocess, img_bytes))
            
        whole = torch.stack(tensors).cuda(non_blocking=True)

        with torch.no_grad(), torch.cuda.amp.autocast():
            feats = self.model.get_image_features(pixel_values=whole)

        out = [v.cpu().tolist() for v in feats]
        return pa.array(out, type=pa.list_(pa.float32(), 512))
    



@gv.udf(
    data_type=pa.list_(pa.float32(), 768),
    cuda=True
)
class EmbedText:
    def __init__(self, column: str):
        self.ready = False
        self.column = column

    def setup(self):
        self.tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")
        self.model = AutoModel.from_pretrained("BAAI/bge-base-en-v1.5").cuda()
        self.ready = True

    def __call__(self, batch: pa.RecordBatch) -> pa.Array:
        if not self.ready:
            self.setup()
        texts = batch.column(self.column).to_pylist()
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        )
        inputs = {k: v.cuda() for k, v in inputs.items()}
        with torch.no_grad():
            out = self.model(**inputs)
            pooled = out.last_hidden_state.mean(dim=1)
        embeddings = pooled.cpu().tolist()
        return pa.array(embeddings, type=pa.list_(pa.float32(), 768))


In [ ]:
table.add_columns({
    "image_embedding": EmbedImage(), 
    "summary_embedding": EmbedText("summary"),
    "occasion_embedding": EmbedText("occasion")
})


In [ ]:
table.backfill("summary_embedding", batch_size=BATCH_SIZE, where="1=1", concurrency=CONCURRENCY )

In [ ]:
table.backfill("occasion_embedding", batch_size=BATCH_SIZE, concurrency=CONCURRENCY, where="1=1")

In [ ]:
table.backfill("image_embedding", batch_size=BATCH_SIZE, concurrency=CONCURRENCY, where="1=1")

In [ ]:
table.search().limit(4).to_pandas()

In [ ]:
table.create_fts_index("summary")
table.create_fts_index("occasion")

In [ ]:
table.create_index(vector_column_name="summary_embedding", num_sub_vectors=128)
table.create_index(vector_column_name="occasion_embedding", num_sub_vectors=128)

In [ ]:
t2 = time.time()
t = t2-t1
t

In [ ]:
import os
import re
import io
import lancedb
import pyarrow as pa
from google import genai
from transformers import CLIPProcessor, CLIPModel
import torch
from typing import List, Optional
import matplotlib.pyplot as plt
from PIL import Image
from lancedb.rerankers import ColbertReranker



genai_client = genai.Client(api_key="...")

# === CONNECT TO LANCEDB ===
db = lancedb.connect(DB_PATH)
tbl = db.open_table("products")

# === CLIP FOR QUERY EMBEDDING ===
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").cuda()
proc = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# === FEATURE DESCRIPTIONS FOR ROUTING ===
FEATURE_DESCRIPTIONS = {
    "summary_embedding": "semantic intent captured from product summary",
    "occasion_embedding": "semantic intent captured from occasion description"
}
VECTOR_FEATURES = set(FEATURE_DESCRIPTIONS.keys())

# === QUERY ROUTER USING GEMINI ===
def choose_feature(query: str) -> str:
    options = "".join(f"- `{f}`: {d}" for f, d in FEATURE_DESCRIPTIONS.items())
    prompt = (
        f"Given the user query:{query}"
        f"Which one of the following features best matches the query intent? "
        f"Choose exactly one option (only return the feature name as plain string without formatting):{options}"
        )
    resp = genai_client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=prompt,
        config={"temperature": 0.0}
    )
    return resp.text.strip()

# === COLOR EXTRACTION ===
COLOR_LIST = ["black","white","red","blue","green","yellow","pink","orange","grey","brown"]
def extract_color(query: str) -> Optional[str]:
    q = query.lower()
    for c in COLOR_LIST:
        if re.search(rf"{c}", q):
            return c
    return None

# === QUERY EMBEDDING FUNCTION ===
def embed_query(query: str) -> List[float]:
    inputs = proc(text=[query], return_tensors="pt", padding=True)
    inputs = {k: v.cuda() for k, v in inputs.items()}
    with torch.no_grad():
        emb = clip.get_text_features(**inputs)[0]
    return emb.cpu().tolist()

reranker_occasion = ColbertReranker(column="occasion")
reranker_summary = ColbertReranker(column="summary")

In [ ]:

# === MAIN SEARCH FUNCTION (Hybrid Only) ===
def search(query: str,
           k: int = 5,
           color_filter: Optional[str] = None) -> List[dict]:
    # Route to feature
    feature = choose_feature(query)
    print(f"[Router] Selected feature: {feature}")

    # Extract color filter
    color = extract_color(query)
    if color:
        print(f"[Filter] Color detected: {color}")

    # Always hybrid search on vector feature
    search_input = embed_query(query)
    reranker = reranker_summary if feature=="summary_embedding" else reranker_occasion
    qb = tbl.search(query_type="hybrid", vector_column_name=feature).vector(search_input).text(query).rerank(reranker)
           
    #qb = tbl.search(search_input, query_type="vector", vector_column_name=feature)
    if color:
        print("filter :", color)
        qb = qb.where(f"color_tags like '{color}'", prefilter=True)

    qb = qb.limit(k*2) # overfetch and rerank
    return qb.to_list()[:k]

# === DISPLAY RESULTS FUNCTION ===
def display_results(results: List[dict], cols: int = 4):
    n = len(results)
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten()
    for ax in axes[n:]:
        ax.axis('off')
    for i, item in enumerate(results):
        img = Image.open(io.BytesIO(item['image_bytes']))
        axes[i].imshow(img)
        axes[i].set_title(item.get('description','')[:15], fontsize=8)
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

# === EXAMPLE USAGE ===
if __name__ == "__main__":
    queries = [
    "black jacket",
    "shirt or t-shirt for business meeting"
    ]
    for q in queries:
        print(f"Query: {q}")
        res = search(q, k=5, color_filter=None)
        print([r["occasion"] for r in  res])
        display_results(res, cols=3)


In [ ]:
import os


import lancedb
from lancedb.pydantic import LanceModel, Vector



class Documents(LanceModel):
    vec: Vector(2) 
    text: str

data = [
    {"text": "rebel spaceships striking from a hidden base", "vec": [0,0]},
    {"text": "have won their first victory against the evil Galactic Empire",  "vec": [0,0]},
    {"text": "during the battle rebel spies managed to steal secret plans",  "vec": [0,0]},
    {"text": "to the Empire's ultimate weapon the Death Star",  "vector": [0,0]},
]
uri = "data/sample-lancedb"
db = lancedb.connect(uri)
table = db.create_table("df", schema=Documents, mode="overwrite")
# ingest docs with auto-vectorization
table.add(data)
# Create a fts index before the hybrid search
table.create_fts_index("text")
# hybrid search with default re-ranker
table.search(query_type="hybrid", vector_column_name="vec").text("text").vector([0,0]).to_pandas()